In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport

pd.set_option('display.max_columns', None) #remove upper limit on the number of columns to display


In [64]:
#load data
data = pd.read_csv('house_train.csv')
test_data = pd.read_csv('house_test.csv')
transaction_id_test = test_data['TRANSACTION_ID']
data_mean_sqm_price = data.groupby("MUNICIPALITY_CODE")["SQM_PRICE"].mean()
data["MUNICIPALITY_MEAN_SQM_PRICE"] = data.apply(
    lambda row: data_mean_sqm_price[row["MUNICIPALITY_CODE"]],
    axis=1
)
test_data["MUNICIPALITY_MEAN_SQM_PRICE"] = test_data.apply(
    lambda row: data_mean_sqm_price[row["MUNICIPALITY_CODE"]],
    axis=1
)


data
test_data


,TRANSACTION_ID,BUILDING_ID,UNIT_ID,FLOOR,MUNICIPALITY_CODE,ZIP_CODE,STREET_CODE,TRADE_DATE,PRICE,SQM_PRICE,CONSTRUCTION_YEAR,REBUILDING_YEAR,AREA_TINGLYST,AREA_RESIDENTIAL,AREA_OTHER,AREA_COMMON_ACCESS_SHARE,AREA_CLOSED_COVER_OUTHOUSE,AREA_OPEN_BALCONY_ROOFTOP,NUMBER_ROOMS,FACILITIES_TOILET,FACILITIES_SHOWER,FACILITIES_KITCHEN,HAS_ELEVATOR,LNG,LAT,DISTANCE_LAKE,DISTANCE_HARBOUR,DISTANCE_COAST,MUNICIPALITY_MEAN_SQM_PRICE
0,f22f66a6-eda2-d049-ea24-09df907e8f38,cf96cec5-f3c6-1b52-6784-c3a25647f8b2,24e6b3bb-29b2-3a1e-7548-622f9c086a08,2,265,4000.0,2656573.0,2024-10-06,NaN,NaN,1962.0,1998.0,71,76,NaN,NaN,NaN,NaN,3,T: Vandskyllende toilet i enheden,V: Badeværelser i enheden,E: Eget køkken med afløb,False,12.094847,55.627949,1415.2,2613.7,2612.8,26822.223099
1,c8cb2732-203f-3b00-4b2b-78ea45ebb5b2,4495a41f-561b-06db-b33c-384a1a83b158,41c02eab-2b4a-3a43-1c92-7dca8398cf54,2,187,2625.0,187106.0,2024-10-04,NaN,NaN,1964.0,NaN,63,68,NaN,NaN,NaN,NaN,2,T: Vandskyllende toilet i enheden,V: Badeværelser i enheden,E: Eget køkken med afløb,False,12.354648,55.652842,871.7,4984.9,4886.3,22081.724044
2,c8f1227e-762c-995d-1315-89bc66f6a9d3,d1a02511-53e8-98e0-55a6-db15d7a71745,3e16a170-88da-794c-d4e6-fe3a218cc33e,3,101,2150.0,1011882.0,2024-10-04,NaN,NaN,2017.0,NaN,69,83,NaN,NaN,NaN,13.0,3,T: Vandskyllende toilet i enheden,V: Badeværelser i enheden,E: Eget køkken med afløb,True,12.598294,55.708251,637.8,23.0,70.3,43109.941311
3,504eb7fd-becf-b3c3-9cc9-53492c7b4bcc,ac0d66d6-3a0a-1cf8-3075-7f9f0dd3e275,c00f9c8a-6cf2-63a9-6c42-a9370bc963ef,3,101,2200.0,101508.0,2024-10-03,NaN,NaN,1892.0,NaN,45,52,NaN,NaN,NaN,5.0,2,T: Vandskyllende toilet i enheden,C: Adgang til badeværelse,E: Eget køkken med afløb,False,12.558023,55.690497,453.1,1935.4,2573.5,43109.941311
4,a1a89478-d718-4df0-cc6c-bec4ef109e19,0e3f916c-85b8-c9bb-5517-18814f1efd8d,366333f0-a55f-4efd-1a7f-90c234b52d5a,2,153,2605.0,15371.0,2024-10-03,NaN,NaN,1967.0,NaN,43,46,NaN,NaN,NaN,NaN,2,T: Vandskyllende toilet i enheden,V: Badeværelser i enheden,E: Eget køkken med afløb,False,12.409968,55.657729,1569.5,4906.3,4983.6,24220.261411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,3eb63142-1df6-79b1-cb69-c78368f46302,48613d71-0d06-3c36-5efd-9e579638acb0,0d87ffc9-41d7-799b-c6ff-f9b0b0b09bc1,st,167,2650.0,167836.0,2015-01-04,NaN,NaN,2007.0,NaN,68,75,NaN,NaN,NaN,NaN,2,T: Vandskyllende toilet i enheden,V: Badeværelser i enheden,E: Eget køkken med afløb,False,12.491772,55.634532,2341.2,1041.3,840.6,26194.995401
4996,b1118c06-b757-9543-6663-c0cb24456524,fd372529-bec3-1611-205b-b3d90b78fb8a,44fba29c-154b-64f6-fcb5-d294372c5d48,2,730,8900.0,7302545.0,2015-01-02,NaN,NaN,1948.0,2003.0,126,142,NaN,NaN,NaN,NaN,4,T: Vandskyllende toilet i enheden,V: Badeværelser i enheden,E: Eget køkken med afløb,False,10.041323,56.459512,1005.6,101.8,347.1,11984.072165
4997,4618d1c9-020a-039c-a5d2-d766a52bfa59,7c32d81e-32c6-21bc-884d-5f92d4a8cb63,e852d1d6-904a-c3f4-8d1f-4662b775cb54,st,101,2700.0,1013480.0,2015-01-02,NaN,NaN,1938.0,NaN,66,71,NaN,5.0,NaN,NaN,2,T: Vandskyllende toilet i enheden,V: Badeværelser i enheden,E: Eget køkken med afløb,False,12.490083,55.706564,361.3,5892.8,6017.2,43109.941311
4998,dc6c8ba2-251c-04c1-7f96-e2aad2559240,3537bd5b-1678-4e5d-6944-81366fb72b89,40a819a0-78e0-6193-7f83-08d83c03a76c,2,101,2720.0,1012264.0,2015-01-01,NaN,NaN,1977.0,NaN,81,92,NaN,NaN,NaN,NaN,3,T: Vandskyllende toilet i enheden,V: Badeværelser i enheden,E: Eget køkken med afløb,False,12.510656,55.694052,1751.5,4565.6,4999.0,43109.941311


In [65]:
#Generates a report for exploratory data analysis and saves it to the folder containing this script
import os
if not os.path.isfile("train_data_report.html"):
    profile_report = ProfileReport(df=data)
    profile_report.to_file("train_data_report.html")

Reading through the report, the main points are:

We have 19 numeric variables, 1 datetime, 1 boolean, 3 text and 4 categorial (28 in total). There's no duplicate data. Approximately 16.2% of the cells are missing.

For ML purposes, we will ignore the following variables (for now):
Transaction id
Building id
Unit id
Longitude
Latitude





In [66]:
def clean_data(data):
    columns_to_drop = ["TRANSACTION_ID", "BUILDING_ID", "UNIT_ID", "SQM_PRICE"]
    try: 
        data = data.drop(columns=columns_to_drop)
    except KeyError:
        pass #error occurs if columns have already been dropped
    
    #For the columns with many missing values, we replace the empty values by the median of the column
    columns_to_fill_missing_values = ["REBUILDING_YEAR", "AREA_OTHER", "AREA_COMMON_ACCESS_SHARE", "AREA_CLOSED_COVER_OUTHOUSE", "AREA_OPEN_BALCONY_ROOFTOP", "ZIP_CODE", "STREET_CODE", "CONSTRUCTION_YEAR"]
    for column in columns_to_fill_missing_values:
        data[column] = data[column].fillna(data[column].median())
    
    data["FLOOR"] = data["FLOOR"].fillna("na")

    #For the columns missing a few values, we choose to drop those few lines (we have plenty of data to work with)
    print(data.isnull().sum(axis = 0))

    # handle datetime and convert it to year
    data['TRADE_DATE'] = pd.to_datetime(data['TRADE_DATE'])
    data['YEAR'] = data['TRADE_DATE'].dt.year
    data['MONTH'] = data['TRADE_DATE'].dt.month
    data = data.drop(columns=['TRADE_DATE'])
    return data

data = clean_data(data)
test_data = clean_data(test_data)

FLOOR                          0
MUNICIPALITY_CODE              0
ZIP_CODE                       0
STREET_CODE                    0
TRADE_DATE                     0
PRICE                          0
CONSTRUCTION_YEAR              0
REBUILDING_YEAR                0
AREA_TINGLYST                  0
AREA_RESIDENTIAL               0
AREA_OTHER                     0
AREA_COMMON_ACCESS_SHARE       0
AREA_CLOSED_COVER_OUTHOUSE     0
AREA_OPEN_BALCONY_ROOFTOP      0
NUMBER_ROOMS                   0
FACILITIES_TOILET              0
FACILITIES_SHOWER              0
FACILITIES_KITCHEN             0
HAS_ELEVATOR                   0
LNG                            0
LAT                            0
DISTANCE_LAKE                  0
DISTANCE_HARBOUR               0
DISTANCE_COAST                 0
MUNICIPALITY_MEAN_SQM_PRICE    0
dtype: int64
FLOOR                             0
MUNICIPALITY_CODE                 0
ZIP_CODE                          0
STREET_CODE                       0
TRADE_DATE        

In [67]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

data.head()
def scale_data(train_data, test_data):
    #we scale all non-binary features to improve model performance
    #we don't normalize price, since that's our y to predict
    standard_scaler = StandardScaler()
    columns_to_feature_scale = ["YEAR", "MONTH", "MUNICIPALITY_MEAN_SQM_PRICE", "CONSTRUCTION_YEAR", "REBUILDING_YEAR", "AREA_TINGLYST", "AREA_RESIDENTIAL", "AREA_OTHER", "AREA_COMMON_ACCESS_SHARE", "AREA_CLOSED_COVER_OUTHOUSE", "AREA_OPEN_BALCONY_ROOFTOP", "NUMBER_ROOMS", "DISTANCE_LAKE", "DISTANCE_HARBOUR", "DISTANCE_COAST"]

    train_data[columns_to_feature_scale] = standard_scaler.fit_transform(train_data[columns_to_feature_scale])
    test_data[columns_to_feature_scale] = standard_scaler.transform(test_data[columns_to_feature_scale])
    #Extract region from munipality code
    #train_data['MUNICIPALITY_CODE'] = train_data['MUNICIPALITY_CODE'].astype(str).str[0]
    #test_data['MUNICIPALITY_CODE'] = test_data['MUNICIPALITY_CODE'].astype(str).str[0]

    encoder = OneHotEncoder(drop='first', sparse_output=False)
    train_encoded = encoder.fit_transform(train_data[['FLOOR', 'FACILITIES_TOILET', 'FACILITIES_SHOWER', 'FACILITIES_KITCHEN']])
    test_encoded = encoder.transform(test_data[['FLOOR', 'FACILITIES_TOILET', 'FACILITIES_SHOWER', 'FACILITIES_KITCHEN']])

    train_encoded_df = pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out())
    test_encoded_df = pd.DataFrame(test_encoded, columns=encoder.get_feature_names_out())

    train_data = pd.concat([train_data.reset_index(drop=True), train_encoded_df.reset_index(drop=True)], axis=1)
    test_data = pd.concat([test_data.reset_index(drop=True), test_encoded_df.reset_index(drop=True)], axis=1)

    train_data = train_data.drop(columns=['FLOOR', 'FACILITIES_TOILET', 'FACILITIES_SHOWER', 'FACILITIES_KITCHEN'])
    test_data = test_data.drop(columns=['FLOOR', 'FACILITIES_TOILET', 'FACILITIES_SHOWER', 'FACILITIES_KITCHEN'])
    
    train_data = train_data.drop(columns=['ZIP_CODE', 'STREET_CODE'], errors='ignore')
    test_data = test_data.drop(columns=['ZIP_CODE', 'STREET_CODE'], errors='ignore')

    return train_data, test_data

data, test_data = scale_data(data, test_data)

#show top 5 rows
data.head()


,MUNICIPALITY_CODE,PRICE,CONSTRUCTION_YEAR,REBUILDING_YEAR,AREA_TINGLYST,AREA_RESIDENTIAL,AREA_OTHER,AREA_COMMON_ACCESS_SHARE,AREA_CLOSED_COVER_OUTHOUSE,AREA_OPEN_BALCONY_ROOFTOP,NUMBER_ROOMS,HAS_ELEVATOR,LNG,LAT,DISTANCE_LAKE,DISTANCE_HARBOUR,DISTANCE_COAST,MUNICIPALITY_MEAN_SQM_PRICE,YEAR,MONTH,FLOOR_10,FLOOR_11,FLOOR_12,FLOOR_13,FLOOR_14,FLOOR_15,FLOOR_16,FLOOR_17,FLOOR_18,FLOOR_19,FLOOR_2,FLOOR_20,FLOOR_21,FLOOR_22,FLOOR_23,FLOOR_24,FLOOR_25,FLOOR_26,FLOOR_27,FLOOR_28,FLOOR_29,FLOOR_3,FLOOR_30,FLOOR_31,FLOOR_32,FLOOR_33,FLOOR_34,FLOOR_35,FLOOR_36,FLOOR_38,FLOOR_39,FLOOR_4,FLOOR_40,FLOOR_41,FLOOR_42,FLOOR_43,FLOOR_5,FLOOR_6,FLOOR_7,FLOOR_8,FLOOR_9,FLOOR_kl,FLOOR_na,FLOOR_st,MUNICIPALITY_CODE_2,MUNICIPALITY_CODE_3,MUNICIPALITY_CODE_4,MUNICIPALITY_CODE_5,MUNICIPALITY_CODE_6,MUNICIPALITY_CODE_7,MUNICIPALITY_CODE_8,FACILITIES_TOILET_B: Intet vandskyllende toilet,FACILITIES_TOILET_T: Vandskyllende toilet i enheden,FACILITIES_SHOWER_D: Hverken badeværelse eller adgang til badeværelse,FACILITIES_SHOWER_V: Badeværelser i enheden,FACILITIES_KITCHEN_F: Adgang til fælles køkken,FACILITIES_KITCHEN_G: Fast kogeinstallation i værelse eller på gang,FACILITIES_KITCHEN_H: Ingen fast kogeinstallation
0,1,4200000.0,0.321784,0.069108,0.532912,0.615859,-0.055133,-0.033663,-0.005372,-0.270200,0.237836,False,12.522463,55.692823,0.272304,0.365470,0.222343,1.265248,1.849094,-0.08305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,1,3425000.0,1.416627,0.069108,-0.722708,-0.532459,-0.055133,-0.033663,-0.005372,-0.128133,0.237836,True,12.556716,55.652070,-0.355159,-0.663611,-0.628456,0.925935,1.849094,-0.08305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,1,4145000.0,-0.472514,-3.205841,-0.244377,-0.222103,-0.055133,-0.033663,-0.005372,-0.128133,0.237836,False,12.529022,55.691044,0.192585,0.242152,0.161167,1.265248,1.849094,-0.08305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,2,2195000.0,0.514992,0.069108,0.712286,0.460681,-0.055133,-0.033663,-0.005372,-0.128133,1.222516,False,12.492201,55.911679,0.615676,0.303946,-0.298599,-1.415407,1.849094,-0.08305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,4,1550000.0,-1.009201,-1.624831,0.383433,-0.097960,-0.055133,0.583210,-0.005372,-0.128133,0.237836,False,10.371957,55.395950,1.807750,-0.329146,-0.342923,-1.103059,1.849094,-0.08305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [68]:
y_train = data["PRICE"]
X_train = data.drop(["PRICE"], axis=1)
y_test = test_data["PRICE"]
X_test = test_data.drop(["PRICE"], axis=1)
train_nan_counts = X_train.isnull().sum()
test_nan_counts = X_test.isnull().sum()

# Display columns with NaN counts greater than 0
print(train_nan_counts[train_nan_counts > 0])
print(test_nan_counts[test_nan_counts > 0])

#imports we'll need
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn import metrics

#First we try out a linear regression model as a start
lin_model = LinearRegression()
lin_model.fit(X_train, y_train)

pred = lin_model.predict(X_test)
scoring = ['neg_mean_absolute_error', 'neg_mean_squared_error', 'r2']
cross_validate(lin_model, X_train, y_train, scoring=scoring, cv=5, return_train_score=False)


Series([], dtype: int64)
Series([], dtype: int64)


{'fit_time': array([0.56038284, 0.57437062, 0.59102917, 0.5253222 , 0.50609589]),
 'score_time': array([0.01765966, 0.02046394, 0.01536822, 0.01685286, 0.01597357]),
 'test_neg_mean_absolute_error': array([-843397.92610737, -709837.28192342, -624222.01682559,
        -610998.97950019, -630238.24843262]),
 'test_neg_mean_squared_error': array([-1.97464138e+12, -1.50836619e+12, -9.53609902e+11, -7.44489544e+11,
        -7.53016664e+11]),
 'test_r2': array([0.68518548, 0.73944326, 0.73555572, 0.7583332 , 0.69602251])}

In [69]:
#Now lets try comparing multiple models:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_validate

models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Random Forest': RandomForestRegressor(),
    'Support Vector Regressor': SVR()
}

results = {}

for model_name, model in models.items():
    print(f"Training {model_name}")
    cv_results = cross_validate(model, X_train, y_train, 
                                 scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'r2'],
                                 cv=2, 
                                 return_train_score=False,
                                 n_jobs=-1)  #Parallel processing to speed up performance, at the cost of making my laptop cry
    
    results[model_name] = {
        'MAE': -cv_results['test_neg_mean_absolute_error'],
        'MSE': -cv_results['test_neg_mean_squared_error'],
        'R-squared': cv_results['test_r2']
    }

results_summary = pd.DataFrame({model_name: {
    'MAE Mean': values['MAE'].mean(),
    'MAE Std': values['MAE'].std(),
    'MSE Mean': values['MSE'].mean(),
    'MSE Std': values['MSE'].std(),
    'R-squared Mean': values['R-squared'].mean(),
    'R-squared Std': values['R-squared'].std()
} for model_name, values in results.items()})

print(results_summary)


Training Linear Regression
Training Ridge Regression
Training Lasso Regression


KeyboardInterrupt: 

                Linear Regression  Ridge Regression  Lasso Regression  \
MAE Mean             3.367269e+05      3.367292e+05      3.367296e+05   
MAE Std              1.123940e+03      1.119435e+03      1.123572e+03   
MSE Mean             3.700087e+11      3.699374e+11      3.699944e+11   
MSE Std              1.891862e+10      1.891951e+10      1.891326e+10   
R-squared Mean       9.166958e-01      9.167119e-01      9.166990e-01   
R-squared Std        3.630585e-03      3.630906e-03      3.629401e-03   

                Random Forest  Support Vector Regressor  
MAE Mean         1.029432e+04              1.352983e+06  
MAE Std          8.773844e+01              2.110435e+03  
MSE Mean         2.198745e+10              4.729721e+12  
MSE Std          9.680805e+09              3.336924e+10  
R-squared Mean   9.950643e-01             -6.521002e-02  
R-squared Std    2.142929e-03              5.428328e-04  

In [70]:
#imports we'll need
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
#Best Parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 500}

#First we try out a linear regression model as a start
random_forest_model = RandomForestRegressor(n_estimators=500,
                                            max_depth=30,
                                            n_jobs=-1)
random_forest_model.fit(X_train, y_train)

pred = random_forest_model.predict(X_test)
scoring = ['neg_mean_absolute_error', 'neg_mean_squared_error', 'r2']
#cross_validate(lin_model, X_train, y_train, scoring=scoring, cv=5, return_train_score=False)


In [17]:
from sklearn.model_selection import cross_validate
from xgboost import XGBRegressor

xgboost_model = XGBRegressor(max_depth=None,               # Adjust max depth as needed
    learning_rate=0.1,            # Typical learning rate for XGBoost
    n_estimators=500,             # Same number of estimators as in Random Forest
    objective='reg:squarederror', # For regression tasks
    subsample=0.8,                # Helps with generalization
    colsample_bytree=0.8,         # Similar to max_features in Random Forest
    n_jobs=-1                     # Use all available CPU cores
)

xgboost_model.fit(X_train, y_train)
pred = xgboost_model.predict(X_test)

scoring = ['neg_mean_absolute_error', 'neg_mean_squared_error', 'r2']

cv_results = cross_validate(xgboost_model, X_train, y_train, scoring=scoring, cv=5, return_train_score=False)

print("Cross-validation results:", cv_results)


Cross-validation results: {'fit_time': array([2.2718153 , 2.58930683, 2.43048191, 2.56911659, 2.46947956]), 'score_time': array([0.06699967, 0.06799865, 0.06360674, 0.06700015, 0.06200004]), 'test_neg_mean_absolute_error': array([-412546.12404386, -460271.83441683, -299778.20187201,
       -311144.16651038, -307043.16240673]), 'test_neg_mean_squared_error': array([-5.69116618e+11, -6.14823053e+11, -3.20019061e+11, -2.56226626e+11,
       -3.08627314e+11]), 'test_r2': array([0.90926648, 0.89379483, 0.91125594, 0.91682695, 0.87541344])}


{'fit_time': array([0.60328627, 0.54752016, 0.55952764, 0.50327873, 0.42204142]),
 'score_time': array([0.01732993, 0.01373267, 0.01605344, 0.01192212, 0.01003551]),
 'test_neg_mean_absolute_error': array([-1066299.35022989,  -997878.16522518,  -789544.47965809,
         -769735.54011854,  -684357.75380477]),
 'test_neg_mean_squared_error': array([-2.79516424e+12, -2.38633470e+12, -1.45288032e+12, -1.16988287e+12,
        -9.61388665e+11]),
 'test_r2': array([0.55437059, 0.58778206, 0.59710371, 0.62024739, 0.61190697])}

In [28]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV


n_samples = 3000
random_indices = np.random.choice(X_train.shape[0], size=n_samples, replace=False)

X_train_reset = X_train.reset_index(drop=True)
y_train_reset = y_train.reset_index(drop=True)

X_train_subset = X_train_reset.iloc[random_indices]
y_train_subset = y_train_reset.iloc[random_indices]

# Define your model, e.g., Random Forest
model = RandomForestRegressor()

# Define the parameter grid (same as before)
param_distributions = {
    'n_estimators': [200, 250, 300, 350],
    'max_features': ['sqrt'],
    'max_depth': [30],
    'bootstrap': [True]}

# Set up RandomizedSearchCV
random_search = GridSearchCV(
    estimator=model,
    param_grid=param_distributions,
    cv=3,  #Mumber of cross-validation folds
    verbose=1,
    n_jobs=-1  #Use all available cores
)

# Fit the model
random_search.fit(X_train_subset, y_train_subset)

# Print the best parameters and score
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)


Fitting 3 folds for each of 4 candidates, totalling 12 fits
Best Parameters: {'bootstrap': True, 'max_depth': 30, 'max_features': 'sqrt', 'n_estimators': 350}
Best Score: 0.7858201187502755


Fitting 3 folds for each of 288 candidates, totalling 864 fits
Best Parameters: {'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 500}
Best Score: 0.7108019589059639


from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV


n_samples = 5000
random_indices = np.random.choice(X_train.shape[0], size=n_samples, replace=False)

X_train_reset = X_train.reset_index(drop=True)
y_train_reset = y_train.reset_index(drop=True)

X_train_subset = X_train_reset.iloc[random_indices]
y_train_subset = y_train_reset.iloc[random_indices]

# Define your model, e.g., Random Forest
model = RandomForestRegressor()

# Define the parameter grid (same as before)
param_distributions = {
    'n_estimators': [50, 100, 200, 500],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Set up RandomizedSearchCV
random_search = GridSearchCV(
    estimator=model,
    param_grid=param_distributions,
    cv=3,  #Mumber of cross-validation folds
    verbose=1,
    n_jobs=-1  #Use all available cores
)

# Fit the model
random_search.fit(X_train_subset, y_train_subset)

# Print the best parameters and score
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)


In [21]:
#This section is for experimenting with more algorithms:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.model_selection import cross_validate
import pandas as pd

n_samples = 20000
random_indices = np.random.choice(X_train.shape[0], size=n_samples, replace=False)

X_train_reset = X_train.reset_index(drop=True)
y_train_reset = y_train.reset_index(drop=True)

X_train_subset = X_train_reset.iloc[random_indices]
y_train_subset = y_train_reset.iloc[random_indices]

# Define the models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Random Forest': RandomForestRegressor(),
    'Support Vector Regressor': SVR(),
    'HistGradientBoosting': HistGradientBoostingRegressor(),
    'XGBoost': XGBRegressor(eval_metric='rmse', use_label_encoder=False),
}

# Initialize results dictionary
results = {}

# Train and cross-validate each model
for model_name, model in models.items():
    print(f"Training {model_name}")
    cv_results = cross_validate(
        model, X_train_subset, y_train_subset,
        scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'r2'],
        cv=3, 
        return_train_score=False,
        n_jobs=-1
    )
    
    results[model_name] = {
        'MAE': -cv_results['test_neg_mean_absolute_error'],
        'MSE': -cv_results['test_neg_mean_squared_error'],
        'R-squared': cv_results['test_r2']
    }

# Summarize results
results_summary = pd.DataFrame({
    model_name: {
        'MAE Mean': values['MAE'].mean(),
        'MAE Std': values['MAE'].std(),
        'MSE Mean': values['MSE'].mean(),
        'MSE Std': values['MSE'].std(),
        'R-squared Mean': values['R-squared'].mean(),
        'R-squared Std': values['R-squared'].std()
    } for model_name, values in results.items()
}).T

print(results_summary)


Training Linear Regression
Training Ridge Regression
Training Lasso Regression
Training Random Forest
Training Support Vector Regressor
Training HistGradientBoosting
Training XGBoost
                              MAE Mean      MAE Std      MSE Mean  \
Linear Regression         7.787645e+05  5760.482911  1.461989e+12   
Ridge Regression          7.791541e+05  5470.901649  1.460676e+12   
Lasso Regression          7.787620e+05  5760.754091  1.461969e+12   
Random Forest             3.878652e+05  6784.753511  5.532518e+11   
Support Vector Regressor  1.341823e+06  5800.800760  4.689497e+12   
HistGradientBoosting      3.869659e+05  5447.066798  5.325378e+11   
XGBoost                   3.778677e+05  3970.448120  4.961298e+11   

                               MSE Std  R-squared Mean  R-squared Std  
Linear Regression         6.136121e+10        0.667169       0.002883  
Ridge Regression          6.130156e+10        0.667470       0.002672  
Lasso Regression          6.135569e+10        0.

                              MAE Mean       MAE Std      MSE Mean  \
Linear Regression         8.529019e+05    607.830480  1.798478e+12   
Ridge Regression          8.529758e+05    590.852778  1.797454e+12   
Lasso Regression          8.529018e+05    607.904049  1.798473e+12   
Random Forest             5.576356e+05   9971.338426  1.053942e+12   
Support Vector Regressor  1.345384e+06   6188.680430  4.800320e+12   
HistGradientBoosting      5.899489e+05  12916.581742  1.065085e+12   
XGBoost                   5.609291e+05   9393.500434  9.874340e+11   

                               MSE Std  R-squared Mean  R-squared Std  
Linear Regression         5.291035e+10        0.601023       0.001420  
Ridge Regression          5.403395e+10        0.601257       0.001617  
Lasso Regression          5.292829e+10        0.601024       0.001424  
Random Forest             9.001448e+10        0.766490       0.014345  
Support Vector Regressor  1.357655e+11       -0.064953       0.005290  
HistGradientBoosting      1.048520e+11        0.764013       0.018443  
XGBoost                   3.081381e+10        0.780952       0.001763  

In [71]:
test_data["PRICE"] = pred
test_data["TRANSACTION_ID"] = transaction_id_test

predictions = test_data[["TRANSACTION_ID", "PRICE"]].to_dict("records")
predictions

[{'TRANSACTION_ID': 'f22f66a6-eda2-d049-ea24-09df907e8f38',
  'PRICE': 2144630.246630954},
 {'TRANSACTION_ID': 'c8cb2732-203f-3b00-4b2b-78ea45ebb5b2',
  'PRICE': 1195719.668629317},
 {'TRANSACTION_ID': 'c8f1227e-762c-995d-1315-89bc66f6a9d3',
  'PRICE': 5353786.844},
 {'TRANSACTION_ID': '504eb7fd-becf-b3c3-9cc9-53492c7b4bcc',
  'PRICE': 3224404.954402539},
 {'TRANSACTION_ID': 'a1a89478-d718-4df0-cc6c-bec4ef109e19',
  'PRICE': 1314364.9375},
 {'TRANSACTION_ID': 'c81137ca-22df-3e18-c1b6-dafdb9c167b2',
  'PRICE': 2318974.6064},
 {'TRANSACTION_ID': '01540048-7515-0588-66a9-63218d129649',
  'PRICE': 3651860.7168888887},
 {'TRANSACTION_ID': '85ad9289-cd67-c681-733f-9ba84e21eb04',
  'PRICE': 1150811.76592166},
 {'TRANSACTION_ID': 'eff60c2b-a8a7-88be-8ee4-6da7e277b5bc',
  'PRICE': 6552559.6190476185},
 {'TRANSACTION_ID': '437587b6-573d-90df-ff33-d8c6e8c016f1',
  'PRICE': 1522415.593096681},
 {'TRANSACTION_ID': 'f95d5c54-8743-6414-24b8-e8810eb963b9',
  'PRICE': 2279584.8888888895},
 {'TRANSACTIO

In [72]:
#Post to Resights API

import requests
r  = requests.post(
    url="https://api.resights.dk/hackathon/avm/ejerlejligheder/v1",
    json={
        "name": "Benjamin Waziri",
        "email": "202005735@post.au.dk",
        "predictions": predictions,
    },
)
print(r.json())

{'accuracy_5_perc': 0.357, 'accuracy_10_perc': 0.622, 'accuracy_20_perc': 0.8842}


linear model performance: {'accuracy_5_perc': 0.1118, 'accuracy_10_perc': 0.2238, 'accuracy_20_perc': 0.4444}
random forest performance: {'accuracy_5_perc': 0.3182, 'accuracy_10_perc': 0.562, 'accuracy_20_perc': 0.8054}
random forest performance (200n): {'accuracy_5_perc': 0.3194, 'accuracy_10_perc': 0.5628, 'accuracy_20_perc': 0.8116}
from now on with lat, lng:
random forest performance (200n): {'accuracy_5_perc': 0.3624, 'accuracy_10_perc': 0.6194, 'accuracy_20_perc': 0.8764}
random forest performance (200n, 30 depth): {'accuracy_5_perc': 0.3606, 'accuracy_10_perc': 0.6216, 'accuracy_20_perc': 0.8786}
Now with month:
random forest performance (200n, 30 depth) {'accuracy_5_perc': 0.3588, 'accuracy_10_perc': 0.624, 'accuracy_20_perc': 0.8808}
random forest performance (500n, 30 depth) {'accuracy_5_perc': 0.3612, 'accuracy_10_perc': 0.6246, 'accuracy_20_perc': 0.8808}
now with region:
random forest performance (500n, 30 depth) {'accuracy_5_perc': 0.3608, 'accuracy_10_perc': 0.6302, 'accuracy_20_perc': 0.8832}